In [2]:
import numpy as np
import astropy as ast
import scipy as sc
import matplotlib.pyplot as plt

import numpy as np
import math
import astropy
import numpy.linalg

In [3]:
position_vector = np.array([3, 3, 3])

In [4]:
rho = np.sqrt(np.sum(position_vector**2))

In [5]:
if rho != 0:
    theta = np.arccos(position_vector[2] / rho)
else:
    theta = np.pi / 2

In [6]:
if position_vector[0] > 0:
    phi = np.arctan(position_vector[1] / position_vector[0])
elif position_vector[0] < 0 :
    phi = np.arctan(position_vector[1] / position_vector[0]) + np.pi
elif position_vector[0] == 0:
    if position_vector[1] > 0:
        phi = np.pi / 2
    if position_vector[1] < 0:
        phi = -np.pi / 2
    if position_vector[1] == 0:
        phi = 0

In [7]:
type(phi)

numpy.float64

In [48]:
rot1 = np.zeros([3,3])

In [49]:
rot1

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [50]:
rot1[0][0] = np.cos(phi)
rot1[0][1] = np.sin(phi)
rot1[1][0] = -np.sin(phi)
rot1[1][1] = np.cos(phi)
rot1[2][2] = 1

In [51]:
rot1

array([[ 0.70710678,  0.70710678,  0.        ],
       [-0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [52]:
rot2 = np.zeros([3,3])

In [53]:
rot2

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [54]:
rot2[0][0] = np.sin(theta)
rot2[0][2] = np.cos(theta)
rot2[2][0] = -np.cos(theta)
rot2[2][2] = np.sin(theta)
rot2[1][1] = 1

In [55]:
rot2

array([[ 0.81649658,  0.        ,  0.57735027],
       [ 0.        ,  1.        ,  0.        ],
       [-0.57735027,  0.        ,  0.81649658]])

In [56]:
leo_tries_1 = rot1 @ position_vector

In [57]:
leo_tries_2 = rot2 @ leo_tries_1

In [58]:
leo_tries_2

array([ 5.19615242e+00,  4.44089210e-16, -4.44089210e-16])

In [59]:
leo_tries_3 = rot2 @ rot1 @ position_vector
leo_tries_3

array([5.19615242e+00, 4.44089210e-16, 0.00000000e+00])

In [60]:
rho

5.196152422706632

In [85]:
ref_L = np.array([0, 0, 0])

In [86]:
if ref_L[1] > 0:
    pseudo_phi = np.arctan(ref_L[2] / ref_L[1])
elif ref_L[1] < 0 :
    pseudo_phi = np.arctan(ref_L[2] / ref_L[1]) + np.pi
elif ref_L[1] == 0:
    if ref_L[2] > 0:
        pseudo_phi = np.pi / 2
    if ref_L[2] < 0:
        pseudo_phi = -np.pi / 2
    if ref_L[2] == 0:
        pseudo_phi = 0

In [87]:
rot3 = np.zeros([3,3])

In [88]:
rot3

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [89]:
rot3[0][0] = 1
rot3[1][1] = np.sin(pseudo_phi)
rot3[1][2] = -np.cos(pseudo_phi)
rot3[2][1] = np.cos(pseudo_phi)
rot3[2][2] = np.sin(pseudo_phi)

In [90]:
rot3

array([[ 1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0.,  1.,  0.]])

In [91]:
leo_tries_4 = rot3 @ leo_tries_3
leo_tries_4

array([5.19615242e+00, 0.00000000e+00, 4.44089210e-16])

In [92]:
rho

5.196152422706632

In [3]:
def get_running_avg_and_std(max_longitude_rad: float, longitudes_rad: np.ndarray, values: np.ndarray,
                            max_value: float, min_value: float) -> tuple:
    """Return the list of longitudes (in degrees) between -max_longitude_rad and max_longitude_rad where 
    there are stars within +/- 2 degrees and their angular moment is between min_value and max_value and the 
    list of average angular momentum in each of those ranges and the list of standard deviations in angular 
    momentum in each of those ranges, given a star cluster's longitudes_rad and values.
    """

    degs = []
    avg_delta_Lz = []
    std_delta_Lz = []

    for deg in np.arange(-max_longitude_rad * u.rad.to(u.deg), max_longitude_rad * u.rad.to(u.deg) + 1):

        delta_Lz_list = []

        for star in range(len(longitudes_rad)):

            if deg - 2 <= longitudes_rad[star] * u.rad.to(u.deg) < deg + 2\
            and min_value <= values[star] <= max_value:
                delta_Lz_list.append(values[star])

        if delta_Lz_list != []:
            degs.append(deg)
            avg_delta_Lz.append(np.mean(delta_Lz_list))
            std_delta_Lz.append(np.std(delta_Lz_list))
            
    return(np.array(degs), avg_delta_Lz, std_delta_Lz)